# SDS Challenge #2 - Used Car Prices

## Problem Statement

Welcome Data Scientist to the 2nd SDS Club Monthly Challenge! This month you have been hired by a new car trader company to help sell its used cars. You will be analyzing used cars from multiple manufacturers and of different models. Your job is to help the car trader company determine the price of its used cars.

## Evaluation

\begin{equation*}
MSE = {\frac{1}{n}\sum_{i=1}^{n}(y_{i} - \hat{y}_{i})^{2}}
\end{equation*}
<br>

## Understanding the Dataset

Each column in the dataset is labeled and explained in more detail below. <br><br>
**manufacturer_name:** the name of the car manufacturer <br>
**model_name:** the name of the car model <br>
**transmission:** the type of transmission the car has <br>
**color:** the body color of the car <br>
**odometer_value:** odometer state in kilometers <br>
**year_produced:** the year the car was produced <br>
**engine_fuel:** the fuel type of the engine of the car <br>
**engine_has_gas:** whether or not the car has a propane tank with tubing <br>
**engine_type:** the engine type of the car <br>
**engine_capacity:** capacity of the engine in liters <br>
**body_type:** the of body the car has <br>
**has_warranty:** whether the car has warranty <br>
**state:** the state of the car (new, owned, etc.) <br>
**drivetrain:** type of drivetrain (front, rear, all) <br>
**feature_1 - feature_9:** these features are boolean values about properties of the car<br>
**duration_listed:** the number of days the car is listed in the catalog <br>
**price_usd:** price of the car in USD <br>

##  Dataset Files
**public_cars.csv** - Dataset to train and analyze <br>
**pred_cars.csv** - Dataset to predict cars prices

## Submission

All submissions should be sent through email to <challenges@superdatascience.com>. When submitting, the file should contain predictions made on the pred_cars.csv file, and it should have the following format:

In [1]:
12000
19500
6000
7500
24000

24000

## Importing the Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importing the dataset and cleaning them

In [3]:
dataset = pd.read_csv("public_cars.csv")
dataset_test = pd.read_csv("pred_cars.csv")

In [4]:
dataset

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,price_usd
0,Ford,Fusion,mechanical,blue,245000,2006,gasoline,False,gasoline,1.6,hatchback,False,owned,front,True,False,False,False,False,False,False,False,False,False,7,4250.0
1,Dodge,Caravan,automatic,silver,265542,2002,gasoline,False,gasoline,3.3,minivan,False,owned,front,False,True,False,False,False,False,False,False,False,False,133,4000.0
2,Ford,Galaxy,mechanical,blue,168000,2009,diesel,False,diesel,1.8,minivan,False,owned,front,False,False,False,True,False,False,True,True,True,True,0,10900.0
3,Mazda,6,mechanical,other,225522,2008,gasoline,False,gasoline,1.8,universal,False,owned,front,False,True,True,False,False,True,False,False,True,True,20,6999.0
4,Audi,80,mechanical,black,370000,1991,gasoline,False,gasoline,1.8,sedan,False,owned,front,False,False,False,False,False,False,False,False,False,True,160,1600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30819,Mazda,Millenia,automatic,green,360493,1999,gasoline,False,gasoline,2.5,sedan,False,owned,front,False,True,False,False,True,True,False,False,False,True,66,2000.0
30820,Audi,A3,automatic,grey,117000,2009,gasoline,False,gasoline,1.4,universal,False,owned,front,False,True,True,True,False,False,False,True,True,True,58,8800.0
30821,Mazda,626,mechanical,black,333000,1997,gasoline,False,gasoline,2.0,hatchback,False,owned,front,False,False,False,False,False,False,False,False,False,True,87,1400.0
30822,Audi,A6,automatic,violet,530000,1995,gasoline,False,gasoline,2.6,universal,False,owned,all,False,True,True,False,False,False,False,False,True,True,52,3500.0


In [5]:
dataset_test

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Renault,Megane,mechanical,blue,360000,1997,gasoline,False,gasoline,1.6,coupe,False,owned,front,False,True,False,False,False,False,False,True,False,True,114
1,Peugeot,206,mechanical,silver,267000,1999,gasoline,False,gasoline,1.4,hatchback,False,owned,front,False,False,False,False,False,False,False,False,False,True,67
2,Volkswagen,Sharan,mechanical,blue,172000,2000,gasoline,False,gasoline,2.0,minivan,False,owned,front,True,False,False,False,False,False,False,False,False,False,50
3,Volvo,XC60,mechanical,white,230000,2009,diesel,False,diesel,2.4,universal,False,owned,front,False,True,True,True,False,True,True,True,True,True,79
4,Mazda,3,mechanical,silver,206000,2007,gasoline,False,gasoline,1.6,sedan,False,owned,front,False,True,False,True,False,True,False,False,False,False,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7702,Chrysler,Sebring,automatic,blue,347618,2001,gasoline,False,gasoline,2.4,sedan,False,owned,front,True,False,False,False,False,False,False,False,False,False,81
7703,Geely,Emgrand 7,automatic,black,9700,2018,gasoline,False,gasoline,2.0,suv,False,owned,front,False,True,False,True,False,True,False,True,True,True,84
7704,Chrysler,Sebring,automatic,red,111111,2001,gasoline,False,gasoline,2.4,sedan,False,owned,front,True,False,False,False,False,False,False,False,False,False,1
7705,Ford,EcoSport,mechanical,white,109000,2016,diesel,False,diesel,1.5,suv,False,owned,front,False,False,False,True,False,False,True,True,True,True,2


## Taking care of missing data

In [6]:
dataset.drop(['manufacturer_name', 'model_name', 'color', 'body_type'],
  axis='columns', inplace=True)
dataset_test.drop(['manufacturer_name', 'model_name', 'color', 'body_type'],
  axis='columns', inplace=True)

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [7]:
dataset

,transmission,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,has_warranty,state,drivetrain,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,price_usd
0,mechanical,245000,2006,gasoline,False,gasoline,1.6,False,owned,front,True,False,False,False,False,False,False,False,False,False,7,4250.0
1,automatic,265542,2002,gasoline,False,gasoline,3.3,False,owned,front,False,True,False,False,False,False,False,False,False,False,133,4000.0
2,mechanical,168000,2009,diesel,False,diesel,1.8,False,owned,front,False,False,False,True,False,False,True,True,True,True,0,10900.0
3,mechanical,225522,2008,gasoline,False,gasoline,1.8,False,owned,front,False,True,True,False,False,True,False,False,True,True,20,6999.0
4,mechanical,370000,1991,gasoline,False,gasoline,1.8,False,owned,front,False,False,False,False,False,False,False,False,False,True,160,1600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30819,automatic,360493,1999,gasoline,False,gasoline,2.5,False,owned,front,False,True,False,False,True,True,False,False,False,True,66,2000.0
30820,automatic,117000,2009,gasoline,False,gasoline,1.4,False,owned,front,False,True,True,True,False,False,False,True,True,True,58,8800.0
30821,mechanical,333000,1997,gasoline,False,gasoline,2.0,False,owned,front,False,False,False,False,False,False,False,False,False,True,87,1400.0
30822,automatic,530000,1995,gasoline,False,gasoline,2.6,False,owned,all,False,True,True,False,False,False,False,False,True,True,52,3500.0


In [8]:
dataset_test

,transmission,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,has_warranty,state,drivetrain,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,mechanical,360000,1997,gasoline,False,gasoline,1.6,False,owned,front,False,True,False,False,False,False,False,True,False,True,114
1,mechanical,267000,1999,gasoline,False,gasoline,1.4,False,owned,front,False,False,False,False,False,False,False,False,False,True,67
2,mechanical,172000,2000,gasoline,False,gasoline,2.0,False,owned,front,True,False,False,False,False,False,False,False,False,False,50
3,mechanical,230000,2009,diesel,False,diesel,2.4,False,owned,front,False,True,True,True,False,True,True,True,True,True,79
4,mechanical,206000,2007,gasoline,False,gasoline,1.6,False,owned,front,False,True,False,True,False,True,False,False,False,False,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7702,automatic,347618,2001,gasoline,False,gasoline,2.4,False,owned,front,True,False,False,False,False,False,False,False,False,False,81
7703,automatic,9700,2018,gasoline,False,gasoline,2.0,False,owned,front,False,True,False,True,False,True,False,True,True,True,84
7704,automatic,111111,2001,gasoline,False,gasoline,2.4,False,owned,front,True,False,False,False,False,False,False,False,False,False,1
7705,mechanical,109000,2016,diesel,False,diesel,1.5,False,owned,front,False,False,False,True,False,False,True,True,True,True,2


In [9]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = "mean")
X_test = dataset_test.values
imputer.fit(X[:, 6:7])
X[:, 6:7] = imputer.transform(X[:, 6:7])
X_test[:, 6:7] = imputer.transform(X_test[:, 6:7])

In [10]:
print(X)
X.shape

[['mechanical' 245000 2006 ... False False 7]
 ['automatic' 265542 2002 ... False False 133]
 ['mechanical' 168000 2009 ... True True 0]
 ...
 ['mechanical' 333000 1997 ... False True 87]
 ['automatic' 530000 1995 ... True True 52]
 ['automatic' 15000 2018 ... True True 75]]


(30824, 21)

In [11]:
print(X_test)
X_test.shape

[['mechanical' 360000 1997 ... False True 114]
 ['mechanical' 267000 1999 ... False True 67]
 ['mechanical' 172000 2000 ... False False 50]
 ...
 ['automatic' 111111 2001 ... False False 1]
 ['mechanical' 109000 2016 ... True True 2]
 ['mechanical' 180000 2000 ... False False 26]]


(7707, 21)

## Encoding categorical data

### Encoding the Independent Variable

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [0, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,18, 19])], remainder = 'passthrough' )
X = np.array(ct.fit_transform(X))
X_test = np.array(ct.transform(X_test))

In [13]:
print(X_test)
X_test.shape

[[0.0 1.0 0.0 ... 1997 1.6 114]
 [0.0 1.0 0.0 ... 1999 1.4 67]
 [0.0 1.0 0.0 ... 2000 2.0 50]
 ...
 [1.0 0.0 0.0 ... 2001 2.4 1]
 [0.0 1.0 1.0 ... 2016 1.5 2]
 [0.0 1.0 0.0 ... 2000 1.4 26]]


(7707, 45)

In [14]:
print(X)
X.shape

[[0.0 1.0 0.0 ... 2006 1.6 7]
 [1.0 0.0 0.0 ... 2002 3.3 133]
 [0.0 1.0 1.0 ... 2009 1.8 0]
 ...
 [0.0 1.0 0.0 ... 1997 2.0 87]
 [1.0 0.0 0.0 ... 1995 2.6 52]
 [1.0 0.0 0.0 ... 2018 3.5 75]]


(30824, 45)

## Feature Scaling

In [15]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)
X_test = sc.fit_transform(X_test)

In [16]:
print(X)

[[-0.71001645  0.71001645 -0.70856972 ...  0.38022628 -0.6805601
  -0.65105271]
 [ 1.40841807 -1.40841807 -0.70856972 ... -0.11675116  1.86649355
   0.46740533]
 [-0.71001645  0.71001645  1.41129372 ...  0.75295936 -0.38090673
  -0.71318927]
 ...
 [-0.71001645  0.71001645 -0.70856972 ... -0.73797296 -0.08125336
   0.05907938]
 [ 1.40841807 -1.40841807 -0.70856972 ... -0.98646168  0.81770675
  -0.25160341]
 [ 1.40841807 -1.40841807 -0.70856972 ...  1.87115859  2.16614693
  -0.04744044]]


In [17]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

## Making the ANN and training it on the Training set

In [18]:
import tensorflow as tf
import time

In [19]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [20]:
!nvidia-smi

Thu Jan  7 21:51:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    28W /  70W |    227MiB / 15079MiB |      7%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
y = y.reshape(len(y), 1)
y.shape

(30824, 1)

In [22]:
X.shape

(30824, 20)

### Initializing the ANN

In [23]:
def build_model():
    with tf.device('/gpu:0'):
      ann = tf.keras.models.Sequential()

      ann.add(tf.keras.layers.Dense(units = 20, activation = 'relu'))

      ann.add(tf.keras.layers.Dense(units = 100, activation = 'relu'))

      ann.add(tf.keras.layers.Dense(units = 100, activation = 'relu'))

      ann.add(tf.keras.layers.Dense(units = 100, activation = 'relu'))

      ann.add(tf.keras.layers.Dense(units = 1, activation = 'relu'))

      start = time.time()
      ann.compile(loss="mean_squared_error", optimizer="adam")
      print("> Compilation Time : ", time.time() - start)
      return ann

In [24]:
ann = build_model()

> Compilation Time :  0.013881206512451172


### Training the ANN model on the Training set

In [25]:
ann.fit( x=X, y=y, batch_size=32, epochs = 100)

Epoch 1/100
964/964 [==============================] - 4s 2ms/step - loss: 35419347.0425
Epoch 2/100
964/964 [==============================] - 2s 2ms/step - loss: 13526712.9368
Epoch 3/100
964/964 [==============================] - 2s 2ms/step - loss: 12503991.2694
Epoch 4/100
964/964 [==============================] - 2s 2ms/step - loss: 12599039.9109
Epoch 5/100
964/964 [==============================] - 2s 2ms/step - loss: 12522081.5223
Epoch 6/100
964/964 [==============================] - 2s 2ms/step - loss: 12661107.8756
Epoch 7/100
964/964 [==============================] - 2s 2ms/step - loss: 12248856.4819
Epoch 8/100
964/964 [==============================] - 2s 2ms/step - loss: 12122467.7326
Epoch 9/100
964/964 [==============================] - 2s 2ms/step - loss: 11415568.3813
Epoch 10/100
964/964 [==============================] - 2s 2ms/step - loss: 11767669.2415
Epoch 11/100
964/964 [==============================] - 2s 2ms/step - loss: 11475401.0912
Epoch 12/100
964/96

### Predicting the results of the Test set

In [26]:
y_pred = ann.predict(X_test)
print(y_pred)

[[ 1555.8259]
 [ 1776.4545]
 [ 2545.0994]
 ...
 [ 3724.1672]
 [12653.0625]
 [ 2010.4209]]


## Building the csv file

In [27]:
df = pd.DataFrame(y_pred, columns=["price_usd"])

df_total = pd.concat([pd.read_csv("pred_cars.csv"), df], axis=1)

print(df_total)

     manufacturer_name model_name  ... duration_listed     price_usd
0              Renault     Megane  ...             114   1555.825928
1              Peugeot        206  ...              67   1776.454468
2           Volkswagen     Sharan  ...              50   2545.099365
3                Volvo       XC60  ...              79  12124.189453
4                Mazda          3  ...              74   4966.831055
...                ...        ...  ...             ...           ...
7702          Chrysler    Sebring  ...              81   3274.054199
7703             Geely  Emgrand 7  ...              84  18957.728516
7704          Chrysler    Sebring  ...               1   3724.167236
7705              Ford   EcoSport  ...               2  12653.062500
7706             Skoda      Fabia  ...              26   2010.420898

[7707 rows x 26 columns]


In [28]:
df_total.to_csv("pred_cars.csv", index=None)